In [1]:
from functools import cache
from pathlib import Path
from time import sleep

import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import table
from tqdm.notebook import tqdm

pd.options.mode.chained_assignment = None

In [2]:
def save_df_as_image(df: pd.DataFrame, path: str) -> None:
    def transform_cell(cell):
        if isinstance(cell, str):
            return cell.replace(" ", "\n")
        return cell

    df_transformed = df.copy().map(transform_cell)

    df_no_index = df_transformed.reset_index(drop=True)

    fig, ax = plt.subplots(
        figsize=(df_no_index.shape[1] * 1.2, df_no_index.shape[0] * 0.5)
    )
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    tbl = table(ax, df_no_index, loc="center", cellLoc="center")

    for key, cell in tbl.get_celld().items():
        if key[1] == -1:
            cell.get_text().set_text("")

    tbl.auto_set_font_size(False)
    tbl.set_fontsize(8)
    tbl.scale(1.5, 1.5)

    plt.tight_layout()
    plt.savefig(path, bbox_inches="tight", dpi=300)
    plt.close(fig)

In [3]:
def write_to_cache(df: pd.DataFrame, file_name: str) -> None:
    df.to_csv(f".cache/{file_name}", index=False)


@cache
def _fetch_year(
    year: int,
    stat: str,
    *,
    base_url: str,
) -> pd.DataFrame:
    base_url = base_url.format(year, stat)

    df = pd.read_html(base_url)[0]

    df = df.drop_duplicates(subset="Player", keep="first")

    df = df[df["G"] > 58]

    if stat == "totals":
        for col in ["PTS", "FTA", "FT", "FG", "FGA"]:
            df[col] = df[col] / df["G"]

    df = df[
        ((df["MP"] < 50) & (df["MP"] > 20))
        | ((df["MP"] >= 50) & (df["MP"] / df["G"] > 20))
    ]

    return df


def fetch_year(
    year: int,
    stat: str = "totals",
    *,
    base_url: str = "https://www.basketball-reference.com/leagues/NBA_{}_{}.html",
) -> tuple[pd.DataFrame, bool]:
    if Path(".cache").exists() is False:
        Path(".cache").mkdir()

    if Path(f".cache/{year}_{stat}.csv").exists():
        return pd.read_csv(f".cache/{year}_{stat}.csv"), True

    df = _fetch_year(year, stat, base_url=base_url)

    write_to_cache(df, f"{year}_{stat}.csv")

    return df, False

In [4]:
league_averages = pd.read_csv("league_averages.csv")
league_averages["Season"] = (
    league_averages["Season"].str.split("-").str[0].astype(int) + 1
)
league_averages

,Rk,Season,Lg,Age,Ht,Wt,G,MP,FG,FGA,...,FG%,3P%,FT%,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,TS%
0,1,2025,NBA,26.3,6-7,215.0,1157,241.2,41.6,89.2,...,0.467,0.360,0.781,98.9,0.543,12.7,25.2,0.191,114.5,0.576
1,2,2024,NBA,26.4,6-7,216.0,1230,241.4,42.2,88.9,...,0.474,0.366,0.784,98.5,0.547,12.1,24.2,0.192,115.3,0.580
2,3,2023,NBA,26.1,6-6,216.0,1230,241.8,42.0,88.3,...,0.475,0.361,0.782,99.2,0.545,12.5,24.0,0.208,114.8,0.581
3,4,2022,NBA,26.1,6-6,215.0,1230,241.4,40.6,88.1,...,0.461,0.354,0.775,98.2,0.532,12.3,23.2,0.192,112.0,0.566
4,5,2021,NBA,26.1,6-6,217.0,1080,241.4,41.2,88.4,...,0.466,0.367,0.778,99.2,0.538,12.4,22.2,0.192,112.3,0.572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,75,1951,NBA,NaN,NaN,NaN,354,NaN,29.8,83.6,...,0.357,NaN,0.733,NaN,0.357,NaN,NaN,0.293,NaN,0.428
75,76,1950,NBA,NaN,NaN,NaN,561,NaN,28.2,83.1,...,0.340,NaN,0.714,NaN,0.340,NaN,NaN,0.284,NaN,0.410
76,77,1949,BAA,NaN,NaN,NaN,360,NaN,29.0,88.7,...,0.327,NaN,0.703,NaN,0.327,NaN,NaN,0.248,NaN,0.390
77,78,1948,BAA,NaN,NaN,NaN,192,NaN,27.2,96.0,...,0.284,NaN,0.675,NaN,0.284,NaN,NaN,0.190,NaN,0.337


# Per Game

In [5]:
per_game_df = pd.DataFrame()

for year in tqdm(league_averages["Season"].unique()):
    league_average_efg = league_averages[league_averages["Season"] == year][
        "eFG%"
    ].values[0]
    league_average_ts = league_averages[league_averages["Season"] == year][
        "TS%"
    ].values[0]

    try:
        _df, read_from_cache = fetch_year(year)
    except Exception as e:
        print(f"Error fetching data for year {year}: {e}")
        continue

    _df["eFG%"] = ((_df["PTS"] - _df["FT"]) / 2) / _df["FGA"]

    _df = _df[_df["PTS"].round(1) >= 29.5]

    # _df["TS%"] = (_df["PTS"] * 0.5) / (_df["FGA"] + (0.475 * _df["FTA"]))
    _df["TS%"] = (_df["PTS"]) / (2 * (_df["FGA"] + (0.44 * _df["FTA"])))

    _df["eFG+%"] = _df["eFG%"] / league_average_efg
    _df["TS+%"] = _df["TS%"] / league_average_ts

    _df["reFG%"] = _df["eFG%"] - league_average_efg
    _df["rTS%"] = _df["TS%"] - league_average_ts

    _df["Year"] = f"{year - 1}-{str(year)[2:]}"

    for col in ["eFG%", "TS%", "eFG+%", "TS+%", "reFG%", "rTS%"]:
        _df[col] = (_df[col].astype(float) * 100).astype(float).round(1)

    _df = (
        _df[["Player", "Year", "PTS", "eFG%", "TS%", "eFG+%", "TS+%", "reFG%", "rTS%"]]
        .sort_values(by="PTS", ascending=False)
        .reset_index(drop=True)
    )

    per_game_df = pd.concat([per_game_df, _df.round(1)], ignore_index=True)

    if read_from_cache is False:
        sleep(5)

per_game_df

  0%|          | 0/79 [00:00<?, ?it/s]

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_10028\2205559678.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  per_game_df = pd.concat([per_game_df, _df.round(1)], ignore_index=True)


Error fetching data for year 1951: HTTP Error 429: Too Many Requests
Error fetching data for year 1950: HTTP Error 429: Too Many Requests
Error fetching data for year 1949: HTTP Error 429: Too Many Requests
Error fetching data for year 1948: HTTP Error 429: Too Many Requests
Error fetching data for year 1947: HTTP Error 429: Too Many Requests


,Player,Year,PTS,eFG%,TS%,eFG+%,TS+%,reFG%,rTS%
0,Shai Gilgeous-Alexander,2024-25,32.7,56.9,63.7,104.7,110.6,2.6,6.1
1,Giannis Antetokounmpo,2024-25,30.4,60.8,62.5,112.0,108.5,6.5,4.9
2,Nikola Jokić,2024-25,29.8,62.5,66.3,115.2,115.0,8.2,8.7
3,Luka Dončić,2023-24,33.9,57.3,61.7,104.7,106.4,2.6,3.7
4,Giannis Antetokounmpo,2023-24,30.4,62.4,64.9,114.0,111.8,7.7,6.9
...,...,...,...,...,...,...,...,...,...
95,Elgin Baylor,1960-61,34.8,43.0,49.8,103.6,106.3,1.5,2.9
96,Oscar Robertson,1960-61,30.5,47.2,55.5,113.9,118.4,5.8,8.6
97,Wilt Chamberlain,1959-60,37.6,46.1,49.3,112.4,106.4,5.1,3.0
98,Jack Twyman,1959-60,31.2,42.2,48.7,102.9,105.3,1.2,2.4


In [6]:
_df = (
    per_game_df.sort_values("eFG%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS", "eFG%"]]
)
save_df_as_image(_df, "eFG.png")
print(_df.to_markdown(index=False))

| Player                | Year    |   PTS |   eFG% |
|:----------------------|:--------|------:|-------:|
| Stephen Curry         | 2015-16 |  30.1 |   63   |
| Nikola Jokić          | 2024-25 |  29.8 |   62.5 |
| Giannis Antetokounmpo | 2023-24 |  30.4 |   62.4 |
| Giannis Antetokounmpo | 2024-25 |  30.4 |   60.8 |
| Stephen Curry         | 2020-21 |  32   |   60.5 |
| Giannis Antetokounmpo | 2019-20 |  29.5 |   58.9 |
| Giannis Antetokounmpo | 2021-22 |  29.9 |   58.2 |
| Kareem Abdul-Jabbar   | 1970-71 |  31.7 |   57.7 |
| Shaquille O'Neal      | 1999-00 |  29.7 |   57.4 |
| Kareem Abdul-Jabbar   | 1971-72 |  34.8 |   57.4 |


In [7]:
_df = (
    per_game_df.sort_values("eFG+%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS", "eFG+%"]]
)
save_df_as_image(_df, "eFG+.png")
print(_df.to_markdown(index=False))

| Player              | Year    |   PTS |   eFG+% |
|:--------------------|:--------|------:|--------:|
| Kareem Abdul-Jabbar | 1970-71 |  31.7 |   128.5 |
| Kareem Abdul-Jabbar | 1971-72 |  34.8 |   126.2 |
| Stephen Curry       | 2015-16 |  30.1 |   125.4 |
| Wilt Chamberlain    | 1965-66 |  33.5 |   124.6 |
| Wilt Chamberlain    | 1960-61 |  38.4 |   122.7 |
| Walt Bellamy        | 1961-62 |  31.6 |   121.8 |
| Kareem Abdul-Jabbar | 1972-73 |  30.2 |   121.5 |
| Wilt Chamberlain    | 1963-64 |  36.8 |   121   |
| Shaquille O'Neal    | 1999-00 |  29.7 |   120.1 |
| Wilt Chamberlain    | 1964-65 |  34.7 |   119.8 |


In [8]:
_df = (
    per_game_df.sort_values("reFG%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS", "reFG%"]]
)
save_df_as_image(_df, "reFG.png")
print(_df.to_markdown(index=False))

| Player              | Year    |   PTS |   reFG% |
|:--------------------|:--------|------:|--------:|
| Stephen Curry       | 2015-16 |  30.1 |    12.8 |
| Kareem Abdul-Jabbar | 1970-71 |  31.7 |    12.8 |
| Kareem Abdul-Jabbar | 1971-72 |  34.8 |    11.9 |
| Wilt Chamberlain    | 1965-66 |  33.5 |    10.7 |
| Kareem Abdul-Jabbar | 1972-73 |  30.2 |     9.8 |
| Shaquille O'Neal    | 1999-00 |  29.7 |     9.6 |
| Wilt Chamberlain    | 1960-61 |  38.4 |     9.4 |
| Walt Bellamy        | 1961-62 |  31.6 |     9.3 |
| Wilt Chamberlain    | 1963-64 |  36.8 |     9.1 |
| Bob McAdoo          | 1973-74 |  30.6 |     8.8 |


In [9]:
_df = (
    per_game_df.sort_values("TS%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS", "TS%"]]
)
save_df_as_image(_df, "TS.png")
print(_df.to_markdown(index=False))

| Player                  | Year    |   PTS |   TS% |
|:------------------------|:--------|------:|------:|
| Stephen Curry           | 2015-16 |  30.1 |  66.9 |
| Nikola Jokić            | 2024-25 |  29.8 |  66.3 |
| Joel Embiid             | 2022-23 |  33.1 |  65.5 |
| Stephen Curry           | 2020-21 |  32   |  65.5 |
| Adrian Dantley          | 1983-84 |  30.6 |  65.2 |
| Giannis Antetokounmpo   | 2023-24 |  30.4 |  64.9 |
| Shai Gilgeous-Alexander | 2024-25 |  32.7 |  63.7 |
| Shai Gilgeous-Alexander | 2023-24 |  30.1 |  63.6 |
| Kevin Durant            | 2013-14 |  32   |  63.5 |
| Giannis Antetokounmpo   | 2021-22 |  29.9 |  63.3 |


In [10]:
_df = (
    per_game_df.sort_values("TS+%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS", "TS+%"]]
)
save_df_as_image(_df, "TS+.png")
print(_df.to_markdown(index=False))

| Player              | Year    |   PTS |   TS+% |
|:--------------------|:--------|------:|-------:|
| Stephen Curry       | 2015-16 |  30.1 |  123.7 |
| Kareem Abdul-Jabbar | 1970-71 |  31.7 |  121.2 |
| Adrian Dantley      | 1983-84 |  30.6 |  120.1 |
| Kareem Abdul-Jabbar | 1971-72 |  34.8 |  119.6 |
| Jerry West          | 1964-65 |  31   |  119.5 |
| Oscar Robertson     | 1963-64 |  31.4 |  118.8 |
| Oscar Robertson     | 1960-61 |  30.5 |  118.4 |
| Bob McAdoo          | 1973-74 |  30.6 |  118.2 |
| Oscar Robertson     | 1966-67 |  30.5 |  118.2 |
| Jerry West          | 1965-66 |  31.3 |  117.6 |


In [11]:
_df = (
    per_game_df.sort_values("rTS%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS", "rTS%"]]
)
save_df_as_image(_df, "rTS.png")
print(_df.to_markdown(index=False))

| Player              | Year    |   PTS |   rTS% |
|:--------------------|:--------|------:|-------:|
| Stephen Curry       | 2015-16 |  30.1 |   12.8 |
| Adrian Dantley      | 1983-84 |  30.6 |   10.9 |
| Kareem Abdul-Jabbar | 1970-71 |  31.7 |   10.6 |
| Kareem Abdul-Jabbar | 1971-72 |  34.8 |    9.9 |
| Kevin Durant        | 2013-14 |  32   |    9.4 |
| Jerry West          | 1964-65 |  31   |    9.3 |
| Adrian Dantley      | 1981-82 |  30.3 |    9.2 |
| Bob McAdoo          | 1973-74 |  30.6 |    9.1 |
| Oscar Robertson     | 1963-64 |  31.4 |    9.1 |
| Oscar Robertson     | 1966-67 |  30.5 |    9   |


# Per 75 Possessions

In [12]:
per_poss_df = pd.DataFrame()

for year in tqdm(league_averages["Season"].unique()):
    league_average_efg = league_averages[league_averages["Season"] == year][
        "eFG%"
    ].values[0]
    league_average_ts = league_averages[league_averages["Season"] == year][
        "TS%"
    ].values[0]

    try:
        _df, read_from_cache = fetch_year(year, "per_poss")
    except Exception as e:
        print(f"Error fetching data for year {year}: {e}")
        continue

    for col in ["PTS", "FTA", "FT", "FG", "FGA"]:
        _df[col] = _df[col] * 0.75

    _df["eFG%"] = ((_df["PTS"] - _df["FT"]) / 2) / _df["FGA"]

    _df = _df[_df["PTS"].round(1) >= 29.5]

    # _df["TS%"] = (_df["PTS"] * 0.5) / (_df["FGA"] + (0.475 * _df["FTA"]))
    _df["TS%"] = (_df["PTS"]) / (2 * (_df["FGA"] + (0.44 * _df["FTA"])))

    _df["eFG+%"] = _df["eFG%"] / league_average_efg
    _df["TS+%"] = _df["TS%"] / league_average_ts

    _df["reFG%"] = _df["eFG%"] - league_average_efg
    _df["rTS%"] = _df["TS%"] - league_average_ts

    _df["Year"] = f"{year - 1}-{str(year)[2:]}"

    for col in ["eFG%", "TS%", "eFG+%", "TS+%", "reFG%", "rTS%"]:
        _df[col] = (_df[col].astype(float) * 100).astype(float).round(1)

    _df = (
        _df[["Player", "Year", "PTS", "eFG%", "TS%", "eFG+%", "TS+%", "reFG%", "rTS%"]]
        .sort_values(by="PTS", ascending=False)
        .reset_index(drop=True)
    )

    per_poss_df = pd.concat([per_poss_df, _df.round(1)], ignore_index=True)

    if read_from_cache is False:
        sleep(5)

per_poss_df = per_poss_df.rename(
    columns={
        "PTS": "PTS/75",
    }
)
per_poss_df

  0%|          | 0/79 [00:00<?, ?it/s]

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_10028\2313195285.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  per_poss_df = pd.concat([per_poss_df, _df.round(1)], ignore_index=True)


Error fetching data for year 1973: HTTP Error 429: Too Many Requests
Error fetching data for year 1972: HTTP Error 429: Too Many Requests
Error fetching data for year 1971: HTTP Error 429: Too Many Requests
Error fetching data for year 1970: HTTP Error 429: Too Many Requests
Error fetching data for year 1969: HTTP Error 429: Too Many Requests
Error fetching data for year 1968: HTTP Error 429: Too Many Requests
Error fetching data for year 1967: HTTP Error 429: Too Many Requests
Error fetching data for year 1966: HTTP Error 429: Too Many Requests
Error fetching data for year 1965: HTTP Error 429: Too Many Requests
Error fetching data for year 1964: HTTP Error 429: Too Many Requests
Error fetching data for year 1963: HTTP Error 429: Too Many Requests
Error fetching data for year 1962: HTTP Error 429: Too Many Requests
Error fetching data for year 1961: HTTP Error 429: Too Many Requests
Error fetching data for year 1960: HTTP Error 429: Too Many Requests
Error fetching data for year 1959:

,Player,Year,PTS/75,eFG%,TS%,eFG+%,TS+%,reFG%,rTS%
0,Shai Gilgeous-Alexander,2024-25,34.4,56.9,63.7,104.7,110.5,2.6,6.1
1,Giannis Antetokounmpo,2024-25,32.3,60.8,62.5,111.9,108.4,6.5,4.9
2,Luka Dončić,2023-24,32.5,57.3,61.7,104.7,106.4,2.6,3.7
3,Shai Gilgeous-Alexander,2023-24,31.8,56.6,63.5,103.5,109.5,1.9,5.5
4,Giannis Antetokounmpo,2023-24,31.2,62.5,65.0,114.3,112.0,7.8,7.0
...,...,...,...,...,...,...,...,...,...
58,Michael Jordan,1988-89,30.0,54.6,61.3,111.6,114.2,5.7,7.6
59,Michael Jordan,1987-88,32.7,53.8,60.4,110.0,112.4,4.9,6.6
60,Dominique Wilkins,1987-88,30.4,47.4,53.3,97.0,99.1,-1.5,-0.5
61,Michael Jordan,1986-87,34.8,48.4,56.2,99.2,104.4,-0.4,2.4


In [13]:
_df = (
    per_poss_df.sort_values("eFG%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS/75", "eFG%"]]
)
save_df_as_image(_df, "eFG75.png")
print(_df.to_markdown(index=False))

| Player                | Year    |   PTS/75 |   eFG% |
|:----------------------|:--------|---------:|-------:|
| Stephen Curry         | 2015-16 |     31.9 |   62.9 |
| Giannis Antetokounmpo | 2023-24 |     31.2 |   62.5 |
| Nikola Jokić          | 2021-22 |     29.8 |   61.9 |
| Giannis Antetokounmpo | 2024-25 |     32.3 |   60.8 |
| Stephen Curry         | 2020-21 |     33   |   60.6 |
| Giannis Antetokounmpo | 2020-21 |     30.1 |   59.9 |
| Giannis Antetokounmpo | 2018-19 |     29.5 |   59.8 |
| Giannis Antetokounmpo | 2019-20 |     33.2 |   58.8 |
| Shaquille O'Neal      | 1994-95 |     30   |   58.4 |
| Shaquille O'Neal      | 1997-98 |     30.1 |   58.3 |


In [14]:
_df = (
    per_poss_df.sort_values("eFG+%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS/75", "eFG+%"]]
)
save_df_as_image(_df, "eFG+75.png")
print(_df.to_markdown(index=False))

| Player                | Year    |   PTS/75 |   eFG+% |
|:----------------------|:--------|---------:|--------:|
| Stephen Curry         | 2015-16 |     31.9 |   125.4 |
| Shaquille O'Neal      | 1997-98 |     30.1 |   122   |
| Shaquille O'Neal      | 1994-95 |     30   |   116.7 |
| Nikola Jokić          | 2021-22 |     29.8 |   116.4 |
| Karl Malone           | 1989-90 |     30.4 |   116   |
| Giannis Antetokounmpo | 2023-24 |     31.2 |   114.3 |
| Giannis Antetokounmpo | 2018-19 |     29.5 |   114   |
| Stephen Curry         | 2020-21 |     33   |   112.6 |
| Michael Jordan        | 1990-91 |     32   |   112.5 |
| Michael Jordan        | 1989-90 |     32   |   112.3 |


In [15]:
_df = (
    per_poss_df.sort_values("reFG%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS/75", "reFG%"]]
)
save_df_as_image(_df, "reFG75.png")
print(_df.to_markdown(index=False))

| Player                | Year    |   PTS/75 |   reFG% |
|:----------------------|:--------|---------:|--------:|
| Stephen Curry         | 2015-16 |     31.9 |    12.7 |
| Shaquille O'Neal      | 1997-98 |     30.1 |    10.5 |
| Nikola Jokić          | 2021-22 |     29.8 |     8.7 |
| Shaquille O'Neal      | 1994-95 |     30   |     8.4 |
| Karl Malone           | 1989-90 |     30.4 |     7.8 |
| Giannis Antetokounmpo | 2023-24 |     31.2 |     7.8 |
| Giannis Antetokounmpo | 2018-19 |     29.5 |     7.4 |
| Stephen Curry         | 2020-21 |     33   |     6.8 |
| Giannis Antetokounmpo | 2024-25 |     32.3 |     6.5 |
| Michael Jordan        | 1990-91 |     32   |     6.1 |


In [16]:
_df = (
    per_poss_df.sort_values("TS%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS/75", "TS%"]]
)
save_df_as_image(_df, "TS75.png")
print(_df.to_markdown(index=False))

| Player                  | Year    |   PTS/75 |   TS% |
|:------------------------|:--------|---------:|------:|
| Stephen Curry           | 2015-16 |     31.9 |  66.9 |
| Nikola Jokić            | 2021-22 |     29.8 |  66   |
| Joel Embiid             | 2022-23 |     35.6 |  65.6 |
| Stephen Curry           | 2020-21 |     33   |  65.5 |
| Giannis Antetokounmpo   | 2023-24 |     31.2 |  65   |
| Giannis Antetokounmpo   | 2018-19 |     29.5 |  64.3 |
| Shai Gilgeous-Alexander | 2024-25 |     34.4 |  63.7 |
| Shai Gilgeous-Alexander | 2023-24 |     31.8 |  63.5 |
| Kevin Durant            | 2013-14 |     31.4 |  63.5 |
| Giannis Antetokounmpo   | 2020-21 |     30.1 |  63.3 |


In [17]:
_df = (
    per_poss_df.sort_values("TS+%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS/75", "TS+%"]]
)
save_df_as_image(_df, "TS+75.png")
print(_df.to_markdown(index=False))

| Player                | Year    |   PTS/75 |   TS+% |
|:----------------------|:--------|---------:|-------:|
| Stephen Curry         | 2015-16 |     31.9 |  123.6 |
| Kevin Durant          | 2013-14 |     31.4 |  117.4 |
| Karl Malone           | 1989-90 |     30.4 |  116.6 |
| Nikola Jokić          | 2021-22 |     29.8 |  116.5 |
| Giannis Antetokounmpo | 2018-19 |     29.5 |  114.9 |
| Stephen Curry         | 2020-21 |     33   |  114.5 |
| Michael Jordan        | 1988-89 |     30   |  114.2 |
| Michael Jordan        | 1990-91 |     32   |  113.3 |
| Isaiah Thomas         | 2016-17 |     31.8 |  113.3 |
| Joel Embiid           | 2022-23 |     35.6 |  112.8 |


In [18]:
_df = (
    per_poss_df.sort_values("rTS%", ascending=False)
    .reset_index(drop=True)
    .head(10)[["Player", "Year", "PTS/75", "rTS%"]]
)
save_df_as_image(_df, "rTS75.png")
print(_df.to_markdown(index=False))

| Player                | Year    |   PTS/75 |   rTS% |
|:----------------------|:--------|---------:|-------:|
| Stephen Curry         | 2015-16 |     31.9 |   12.8 |
| Nikola Jokić          | 2021-22 |     29.8 |    9.4 |
| Kevin Durant          | 2013-14 |     31.4 |    9.4 |
| Karl Malone           | 1989-90 |     30.4 |    8.9 |
| Giannis Antetokounmpo | 2018-19 |     29.5 |    8.3 |
| Stephen Curry         | 2020-21 |     33   |    8.3 |
| Michael Jordan        | 1988-89 |     30   |    7.6 |
| Joel Embiid           | 2022-23 |     35.6 |    7.5 |
| Isaiah Thomas         | 2016-17 |     31.8 |    7.3 |
| Michael Jordan        | 1990-91 |     32   |    7.1 |


In [19]:
with pd.ExcelWriter("output.xlsx") as writer:
    per_game_df.sort_values(["Player", "Year"]).reset_index(drop=True).to_excel(
        writer, sheet_name="per_game", index=False
    )
    per_poss_df.sort_values(["Player", "Year"]).reset_index(drop=True).to_excel(
        writer, sheet_name="per_poss", index=False
    )

# Aggregations

In [20]:
least_per_game = per_game_df.copy()

stat_cols = [col for col in least_per_game.columns if col not in ["Player", "Year"]]

ranks = least_per_game[stat_cols].rank(ascending=False)

least_per_game["avg_rank"] = ranks.mean(axis=1)

print(least_per_game.sort_values("avg_rank", ascending=True)[["Player", "Year", *stat_cols]].reset_index(drop=True).head(10).to_markdown(index=False))
print("\n\n\n")
print(least_per_game.sort_values("avg_rank", ascending=False)[["Player", "Year", *stat_cols]].reset_index(drop=True).head(10).to_markdown(index=False))

| Player              | Year    |   PTS |   eFG% |   TS% |   eFG+% |   TS+% |   reFG% |   rTS% |
|:--------------------|:--------|------:|-------:|------:|--------:|-------:|--------:|-------:|
| Kareem Abdul-Jabbar | 1971-72 |  34.8 |   57.4 |  60.3 |   126.2 |  119.6 |    11.9 |    9.9 |
| Kareem Abdul-Jabbar | 1970-71 |  31.7 |   57.7 |  60.6 |   128.5 |  121.2 |    12.8 |   10.6 |
| Stephen Curry       | 2015-16 |  30.1 |   63   |  66.9 |   125.4 |  123.7 |    12.8 |   12.8 |
| Stephen Curry       | 2020-21 |  32   |   60.5 |  65.5 |   112.5 |  114.5 |     6.7 |    8.3 |
| Kevin Durant        | 2013-14 |  32   |   56   |  63.5 |   111.7 |  117.3 |     5.9 |    9.4 |
| Karl Malone         | 1989-90 |  31   |   56.7 |  62.6 |   115.9 |  116.6 |     7.8 |    8.9 |
| Adrian Dantley      | 1983-84 |  30.6 |   55.8 |  65.2 |   112.7 |  120.1 |     6.3 |   10.9 |
| Adrian Dantley      | 1981-82 |  30.3 |   57   |  63.1 |   115.2 |  117.1 |     7.5 |    9.2 |
| Nikola Jokić        | 2024-2

In [21]:
least_per_poss = per_poss_df.copy()

stat_cols = [col for col in least_per_poss.columns if col not in ["Player", "Year"]]

ranks = least_per_poss[stat_cols].rank(ascending=False)

least_per_poss["avg_rank"] = ranks.mean(axis=1)

print(least_per_poss.sort_values("avg_rank", ascending=True)[["Player", "Year", *stat_cols]].reset_index(drop=True).head(10).to_markdown(index=False))
print("\n\n\n")
print(least_per_poss.sort_values("avg_rank", ascending=False)[["Player", "Year", *stat_cols]].reset_index(drop=True).head(10).to_markdown(index=False))

| Player                | Year    |   PTS/75 |   eFG% |   TS% |   eFG+% |   TS+% |   reFG% |   rTS% |
|:----------------------|:--------|---------:|-------:|------:|--------:|-------:|--------:|-------:|
| Stephen Curry         | 2015-16 |     31.9 |   62.9 |  66.9 |   125.4 |  123.6 |    12.7 |   12.8 |
| Stephen Curry         | 2020-21 |     33   |   60.6 |  65.5 |   112.6 |  114.5 |     6.8 |    8.3 |
| Nikola Jokić          | 2021-22 |     29.8 |   61.9 |  66   |   116.4 |  116.5 |     8.7 |    9.4 |
| Giannis Antetokounmpo | 2023-24 |     31.2 |   62.5 |  65   |   114.3 |  112   |     7.8 |    7   |
| Joel Embiid           | 2022-23 |     35.6 |   57.5 |  65.6 |   105.4 |  112.8 |     3   |    7.5 |
| Karl Malone           | 1989-90 |     30.4 |   56.7 |  62.6 |   116   |  116.6 |     7.8 |    8.9 |
| Kevin Durant          | 2013-14 |     31.4 |   56.1 |  63.5 |   111.9 |  117.4 |     6   |    9.4 |
| Giannis Antetokounmpo | 2021-22 |     32.7 |   58.1 |  63.3 |   109.2 |  111.8 |